## Previsão Ibovespa
Esse notebook tem como objetivo realizar a previsão do índice IBovespa utilizando alguns modelos de Machine Learning. Esse projeto foi realizado como parte da Fase 2 da pós graduação em Data Analytics da POSTECH FIAP.

### 1. Entendimento do Negócio
O problema
Imagine que você foi escalado para um time de investimentos e precisará realizar um modelo preditivo com dados da IBOVESPA (Bolsa de valores) para criar uma série temporal e prever diariamente o fechamento da base.

Você precisará demonstrar para o time de investimentos:
1. O modelo com o storytelling, desde a captura do dado até a entrega do
modelo;
2. Justificar a técnica utilizada;
3. Atingir uma acuracidade adequada (acima de 70%)

### 2. Entendimento dos Dados
Para isso, utilize a base de dados contida no site da Investing https://br.investing.com/indices/bovespa-historical-data) e selecione o período “diário”, com o intervalo de tempo que achar adequado.

Testaremos alguns períodos Históricos para treinamento com uso de 180 dias anteriores para previsão.

In [1]:
!pip install requests beautifulsoup4 pandas

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [41]:
# web scraping
url = "https://br.investing.com/indices/bovespa-historical-data"
response = requests.get(url)

In [42]:
# transforma o conteúdo HTML de uma página da web em um objeto Python
soup = BeautifulSoup(response.content, 'html.parser')

In [43]:
# Encontrar a tabela na página HTML
#table = soup.find('table', {'class': 'genTbl closedTbl historicalTbl'})
table = soup.find('table')

In [44]:
# Encontrar os cabeçalhos
headers = [th.text for th in table.find_all('th')]

In [45]:
# Encontrar os dados
data = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
    data.append([col.text for col in cols])

In [46]:
# Imprimir os resultados
print("Cabeçalhos:", headers)
print("Dados Atuais:")
for row in data:
    print(row)

Cabeçalhos: ['', '', '', '', '', '', '']
Dados Atuais:
[]
['10.01.2025', '118.856', '119.781', '120.052', '118.732', '9,26M', '-0.77%']
['10.01.2025', '118.856', '119.781', '120.052', '118.732', '9,26M', '-0.77%']
['09.01.2025', '119.781', '119.625', '120.145', '119.502', '6,61M', '+0.13%']
['09.01.2025', '119.781', '119.625', '120.145', '119.502', '6,61M', '+0.13%']
['08.01.2025', '119.625', '121.160', '121.160', '119.351', '10,23M', '-1.27%']
['08.01.2025', '119.625', '121.160', '121.160', '119.351', '10,23M', '-1.27%']
['07.01.2025', '121.163', '120.022', '121.713', '120.022', '11,12M', '+0.95%']
['07.01.2025', '121.163', '120.022', '121.713', '120.022', '11,12M', '+0.95%']
['06.01.2025', '120.022', '118.534', '120.322', '118.534', '9,69M', '+1.26%']
['06.01.2025', '120.022', '118.534', '120.322', '118.534', '9,69M', '+1.26%']
['03.01.2025', '118.533', '120.125', '120.356', '118.404', '9,80M', '-1.33%']
['03.01.2025', '118.533', '120.125', '120.356', '118.404', '9,80M', '-1.33%']
['

In [47]:
df = pd.DataFrame(data, columns=['Data','Último','Abertura','Máxima','Mínima','Vol.', 'Var%'])

In [48]:
df = df.drop(index=0)

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 1 to 30
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Data      30 non-null     object
 1   Último    30 non-null     object
 2   Abertura  30 non-null     object
 3   Máxima    30 non-null     object
 4   Mínima    30 non-null     object
 5   Vol.      30 non-null     object
 6   Var%      30 non-null     object
dtypes: object(7)
memory usage: 1.8+ KB


In [50]:
df.head(3)

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
1,10.01.2025,118.856,119.781,120.052,118.732,"9,26M",-0.77%
2,10.01.2025,118.856,119.781,120.052,118.732,"9,26M",-0.77%
3,09.01.2025,119.781,119.625,120.145,119.502,"6,61M",+0.13%


In [55]:
# Convertendo os valores para float e substituindo o ponto por vírgula
df['Último'] = df['Último'].astype(float)
df['Abertura'] = df['Abertura'].astype(float)
df['Máxima'] = df['Máxima'].astype(float)
df['Mínima'] = df['Mínima'].astype(float)
df['Var%'] = df['Var%'].str.replace('.', ',')

In [56]:
df.head(3)

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
1,10.01.2025,118.856,119.781,120.052,118.732,"9,26M","-0,77%"
2,10.01.2025,118.856,119.781,120.052,118.732,"9,26M","-0,77%"
3,09.01.2025,119.781,119.625,120.145,119.502,"6,61M","+0,13%"


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 1 to 30
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Data      30 non-null     object 
 1   Último    30 non-null     float64
 2   Abertura  30 non-null     float64
 3   Máxima    30 non-null     float64
 4   Mínima    30 non-null     float64
 5   Vol.      30 non-null     object 
 6   Var%      30 non-null     object 
dtypes: float64(4), object(3)
memory usage: 1.8+ KB


In [14]:
df.shape

(30, 7)

In [58]:
# Teste para ver como estão os dados no Excel
#df.to_excel('dados_ibovespa.xlsx', index=False)

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from datetime import datetime


from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

In [16]:
url = 'https://raw.githubusercontent.com/Data-Analitycs-Pos-Tech-Fiap/Ibovespa-prev/refs/heads/main/datasets/Dados%20Hist%C3%B3ricos%20-%20Ibovespa%202000%20a%202025.csv'

dados_030125 = pd.read_csv(url, sep=',', encoding='utf-8')

dados_030125.head(3)

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,03.01.2025,118.533,120.125,120.356,118.404,"9,80B","-1,33%"
1,02.01.2025,120.125,120.283,120.782,119.120,"9,37B","-0,13%"
2,30.12.2024,120.283,120.267,121.050,120.158,"8,90B","0,01%"


In [17]:
# compreendendo os tipos de dados e o shape da base
print(dados_030125.info())
print('----------------------------------------------------')
print('TAMANHO DA BASE: ', dados_030125.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4954 entries, 0 to 4953
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Data      4954 non-null   object 
 1   Último    4954 non-null   float64
 2   Abertura  4954 non-null   float64
 3   Máxima    4954 non-null   float64
 4   Mínima    4954 non-null   float64
 5   Vol.      4953 non-null   object 
 6   Var%      4954 non-null   object 
dtypes: float64(4), object(3)
memory usage: 271.0+ KB
None
----------------------------------------------------
TAMANHO DA BASE:  (4954, 7)


In [59]:
# Concatenando os DataFrames verticalmente
df_concatenado = pd.concat([df, dados_030125], ignore_index=True)

print(df_concatenado)

            Data   Último  Abertura   Máxima   Mínima     Vol.    Var%
0     10.01.2025  118.856   119.781  120.052  118.732    9,26M  -0,77%
1     10.01.2025  118.856   119.781  120.052  118.732    9,26M  -0,77%
2     09.01.2025  119.781   119.625  120.145  119.502    6,61M  +0,13%
3     09.01.2025  119.781   119.625  120.145  119.502    6,61M  +0,13%
4     08.01.2025  119.625   121.160  121.160  119.351   10,23M  -1,27%
...          ...      ...       ...      ...      ...      ...     ...
4979  10.01.2005   24.292    24.747   24.825   24.086  126,23M  -1,84%
4980  07.01.2005   24.747    24.376   24.874   24.376  104,96M   1,56%
4981  06.01.2005   24.367    24.695   24.808   24.250  151,17M  -1,32%
4982  05.01.2005   24.692    24.859   25.001   24.523  127,42M  -0,63%
4983  04.01.2005   24.848    25.722   25.873   24.791  146,14M  -3,40%

[4984 rows x 7 columns]


In [60]:
# Verificando duplicidades na coluna 'Data'
duplicados = df_concatenado['Data'].duplicated()
print(duplicados)
# Contando o número de duplicatas
print(duplicados.sum())

0       False
1        True
2       False
3        True
4       False
        ...  
4979    False
4980    False
4981    False
4982    False
4983    False
Name: Data, Length: 4984, dtype: bool
25


In [61]:
# Removendo as linhas duplicadas e atribuindo o resultado a um novo DataFrame
df_sem_duplicadas = df_concatenado.drop_duplicates(subset='Data', keep='first')

In [62]:
df_sem_duplicadas.head(3)

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,10.01.2025,118.856,119.781,120.052,118.732,"9,26M","-0,77%"
2,09.01.2025,119.781,119.625,120.145,119.502,"6,61M","+0,13%"
4,08.01.2025,119.625,121.160,121.160,119.351,"10,23M","-1,27%"


In [63]:
# Verificando duplicidades na coluna 'Data'
duplicados2 = df_sem_duplicadas['Data'].duplicated()
print(duplicados2)
# Contando o número de duplicatas
print(duplicados2.sum())

0       False
2       False
4       False
6       False
8       False
        ...  
4979    False
4980    False
4981    False
4982    False
4983    False
Name: Data, Length: 4959, dtype: bool
0


In [64]:
dados = df_sem_duplicadas

In [66]:
dados.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,10.01.2025,118.856,119.781,120.052,118.732,"9,26M","-0,77%"
2,09.01.2025,119.781,119.625,120.145,119.502,"6,61M","+0,13%"
4,08.01.2025,119.625,121.160,121.160,119.351,"10,23M","-1,27%"
6,07.01.2025,121.163,120.022,121.713,120.022,"11,12M","+0,95%"
8,06.01.2025,120.022,118.534,120.322,118.534,"9,69M","+1,26%"


In [67]:
dados.columns

Index(['Data', 'Último', 'Abertura', 'Máxima', 'Mínima', 'Vol.', 'Var%'], dtype='object')

In [68]:
# Teste para ver como estão os dados no Excel
#dados.to_excel('dados_ibovespa.xlsx', index=False)

In [71]:
#df['Var%'] = df['Mínima'].astype(float)
dados['Var%'] = dados['Var%'].str.replace('%', '')

<ipython-input-71-75bd05a71db1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados['Var%'] = dados['Var%'].str.replace('%', '')


In [72]:
dados.head(3)

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,10.01.2025,118.856,119.781,120.052,118.732,"9,26M","-0,77"
2,09.01.2025,119.781,119.625,120.145,119.502,"6,61M","+0,13"
4,08.01.2025,119.625,121.160,121.160,119.351,"10,23M","-1,27"


In [76]:
dados['Var%'] = dados['Var%'].str.replace(',', '.').astype(float)

AttributeError: Can only use .str accessor with string values!

In [75]:
dados.head(3)

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,10.01.2025,118.856,119.781,120.052,118.732,"9,26M",-0.77
2,09.01.2025,119.781,119.625,120.145,119.502,"6,61M",0.13
4,08.01.2025,119.625,121.160,121.160,119.351,"10,23M",-1.27


In [77]:
# Teste para ver como estão os dados no Excel
#dados.to_excel('dados_ibovespa.xlsx', index=False)

In [78]:
# Verificar valores ausentes
print(dados.isnull().sum())

Data        0
Último      0
Abertura    0
Máxima      0
Mínima      0
Vol.        1
Var%        0
dtype: int64


In [79]:
# Tratar valores ausentes
dados.fillna(method='ffill', inplace=True)

<ipython-input-79-70ce5f8066d4>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados.fillna(method='ffill', inplace=True)
<ipython-input-79-70ce5f8066d4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados.fillna(method='ffill', inplace=True)


In [80]:
# Converter coluna de datas (se necessário)
dados['Data'] = pd.to_datetime(dados['Data'], format='%d.%m.%Y')

<ipython-input-80-91a0dfb5f86a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados['Data'] = pd.to_datetime(dados['Data'], format='%d.%m.%Y')


In [ ]:
# Convertendo a coluna 'Data' para datetime
#dados.index = pd.to_datetime(dados.index, format='%d.%m.%Y')

# Ordenar o índice em ordem crescente
#dados = dados.sort_index()

#dados = dados.asfreq('D', method='pad')
#dados.reset_index(inplace=True)

# preenchendo o restante das colunas com o último valor válido
#for column in dados.columns:
#    dados[column].fillna(dados[column].iloc[-1], inplace=True)

# verificando o resultado
#dados.head(10)


In [ ]:
# Configurar índice
dados.set_index('Data', inplace=True)

In [ ]:
# incluindo coluna categórica
dados['unique_id'] = 'ibovespa'

In [ ]:
# renomeando colunas
dados = dados.rename(columns={'Data': 'ds', 'Último': 'y'})
# selecionando colunas
dados = dados[['ds', 'y', 'unique_id']]
dados.head(10)

In [ ]:
# df com data como índice e valores só coluna y
df1 = dados[['ds', 'y']]
df1 = dados.set_index('ds')

In [ ]:
dados.head(1)

In [ ]:
plt.plot(df1.index, df1["y"])
plt.xlabel('Data')
plt.ylabel('Valor')
plt.title('Ibovespa diário')
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Decompose the time series
resultados = seasonal_decompose(df1['y'], model='additive', period=1)


In [ ]:
fig, (ax1,ax2,ax3,ax4) = plt.subplots(4,1, figsize = (15,10))

# mostrando a série do data frame
resultados.observed.plot(ax=ax1)
# mostrando a tendência dos dados
resultados.trend.plot(ax=ax2)
# mostrando a sazonalidade - constante e recorrente
resultados.seasonal.plot(ax=ax3)
# mostrando os resíduos
resultados.resid.plot(ax=ax4)

plt.tight_layout()

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
X = df1.y.values

In [ ]:
result = adfuller(X)

# como o teste estatístico é maior que os valores críticos e temos um p-value alto, descartamos a hipótese de série estacionária

print("Teste ADF")
print(f"Teste Estatístico: {result[0]}")
print(f"P-Value: {result[1]}")
print("Valores críticos:")

for key, value in result[4].items():
  print(f"\t{key}: {value}")

In [ ]:
df1.shape

In [ ]:
# Calcular média móvel com janela menor
ma = df1['y'].rolling(window=30).mean()

# Criar o gráfico
f, ax = plt.subplots(figsize=(10,6))
df1['y'].plot(ax=ax, label='Original')
ma.plot(ax=ax, color='r', label='Média Móvel')
plt.legend()
plt.tight_layout()




In [ ]:
# Verificar o DataFrame df1
print("Informações do DataFrame:")
print(df1.info())
print("\nPrimeiras linhas do DataFrame:")
print(df1.head())

# Verificar a série da média móvel
print("\nInformações da média móvel:")
print(ma.head())
print("\nVerificar valores nulos na média móvel:")
print(ma.isnull().sum())
